# Limpieza y preparación de datos

In [1]:
# Importación de librerías

# Librerías para manipulación de datos
import pandas as pd
import re
from datetime import datetime, timedelta

# Librerías para visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Librerías para preprocesamiento de datos
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.utils import resample

# Librerías para división y manejo de datos
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

# Librerías para modelos predictivos
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# Librerías para validación cruzada y métricas
from sklearn.model_selection import cross_validate, KFold
from sklearn.metrics import (make_scorer, precision_score, recall_score, f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, roc_auc_score, roc_curve, accuracy_score)

# Otras librerías
import numpy as np

In [2]:
# Ruta de los datos
platform_path = '/proyecto/datasets/final_provider/'

# Cargar los datasets
contract = pd.read_csv(platform_path + 'contract.csv')
personal = pd.read_csv(platform_path + 'personal.csv')
internet = pd.read_csv(platform_path + 'internet.csv')
phone = pd.read_csv(platform_path + 'phone.csv')

# Confirmar la carga mostrando 5 filas de cada dataset
print("Contract Dataset:")
print(contract.head())

print("\nPersonal Dataset:")
print(personal.head())

print("\nInternet Dataset:")
print(internet.head())

print("\nPhone Dataset:")
print(phone.head())

Contract Dataset:
   customerID   BeginDate              EndDate            Type  \
0  7590-VHVEG  2020-01-01                   No  Month-to-month   
1  5575-GNVDE  2017-04-01                   No        One year   
2  3668-QPYBK  2019-10-01  2019-12-01 00:00:00  Month-to-month   
3  7795-CFOCW  2016-05-01                   No        One year   
4  9237-HQITU  2019-09-01  2019-11-01 00:00:00  Month-to-month   

  PaperlessBilling              PaymentMethod  MonthlyCharges TotalCharges  
0              Yes           Electronic check           29.85        29.85  
1               No               Mailed check           56.95       1889.5  
2              Yes               Mailed check           53.85       108.15  
3               No  Bank transfer (automatic)           42.30      1840.75  
4              Yes           Electronic check           70.70       151.65  

Personal Dataset:
   customerID  gender  SeniorCitizen Partner Dependents
0  7590-VHVEG  Female              0     Yes    

In [3]:
# Unir los datasets utilizando la columna customerID como clave primaria
merged_data = contract.merge(personal, on='customerID', how='inner') \
                      .merge(internet, on='customerID', how='inner') \
                      .merge(phone, on='customerID', how='inner')

# Confirmar la unión exitosa
print(f"Dimensiones del dataset combinado: {merged_data.shape}")
print()
print(merged_data.head())

Dimensiones del dataset combinado: (4835, 20)

   customerID   BeginDate              EndDate            Type  \
0  5575-GNVDE  2017-04-01                   No        One year   
1  3668-QPYBK  2019-10-01  2019-12-01 00:00:00  Month-to-month   
2  9237-HQITU  2019-09-01  2019-11-01 00:00:00  Month-to-month   
3  9305-CDSKC  2019-03-01  2019-11-01 00:00:00  Month-to-month   
4  1452-KIOVK  2018-04-01                   No  Month-to-month   

  PaperlessBilling            PaymentMethod  MonthlyCharges TotalCharges  \
0               No             Mailed check           56.95       1889.5   
1              Yes             Mailed check           53.85       108.15   
2              Yes         Electronic check           70.70       151.65   
3              Yes         Electronic check           99.65        820.5   
4              Yes  Credit card (automatic)           89.10       1949.4   

   gender  SeniorCitizen Partner Dependents InternetService OnlineSecurity  \
0    Male            

In [4]:
# Manejar valores ausentes e inconsistencias para obtener un dataset limpio y confiable 

# Renombrar correctamente columnas con formato CamelCase
merged_data.rename(columns={'CustomerID': 'CustomerId', 'gender': 'Gender'}, inplace=True)

# Verificar los nuevos nombres de las columnas
print("Nuevos nombres de las columnas:")
print(merged_data.columns)

Nuevos nombres de las columnas:
Index(['customerID', 'BeginDate', 'EndDate', 'Type', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Gender',
       'SeniorCitizen', 'Partner', 'Dependents', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'MultipleLines'],
      dtype='object')


In [5]:
# Visualizar las primeras 10 filas del dataset unido con columnas corregidas
merged_data.head(10)

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Gender,SeniorCitizen,Partner,Dependents,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,MultipleLines
0,5575-GNVDE,2017-04-01,No,One year,No,Mailed check,56.95,1889.5,Male,0,No,No,DSL,Yes,No,Yes,No,No,No,No
1,3668-QPYBK,2019-10-01,2019-12-01 00:00:00,Month-to-month,Yes,Mailed check,53.85,108.15,Male,0,No,No,DSL,Yes,Yes,No,No,No,No,No
2,9237-HQITU,2019-09-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,70.70,151.65,Female,0,No,No,Fiber optic,No,No,No,No,No,No,No
3,9305-CDSKC,2019-03-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,99.65,820.5,Female,0,No,No,Fiber optic,No,No,Yes,No,Yes,Yes,Yes
4,1452-KIOVK,2018-04-01,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.4,Male,0,No,Yes,Fiber optic,No,Yes,No,No,Yes,No,Yes
5,7892-POOKP,2017-07-01,2019-11-01 00:00:00,Month-to-month,Yes,Electronic check,104.80,3046.05,Female,0,Yes,No,Fiber optic,No,No,Yes,Yes,Yes,Yes,Yes
6,6388-TABGU,2014-12-01,No,One year,No,Bank transfer (automatic),56.15,3487.95,Male,0,No,Yes,DSL,Yes,Yes,No,No,No,No,No
7,9763-GRSKD,2019-01-01,No,Month-to-month,Yes,Mailed check,49.95,587.45,Male,0,Yes,Yes,DSL,Yes,No,No,No,No,No,No
8,8091-TTVAX,2015-04-01,No,One year,No,Credit card (automatic),100.35,5681.1,Male,0,Yes,No,Fiber optic,No,No,Yes,No,Yes,Yes,Yes
9,0280-XJGEX,2015-09-01,2019-10-01 00:00:00,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Male,0,No,No,Fiber optic,No,Yes,Yes,No,Yes,Yes,Yes


In [6]:
# Verificar tipo de datos en las columnas del dataset merged_data

print(merged_data.dtypes)

customerID           object
BeginDate            object
EndDate              object
Type                 object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
MultipleLines        object
dtype: object


In [7]:
# Cambiar tipo de datos en columna 'BeginDate'(object) a datetime y con un formato que coincida con el dataset
merged_data['BeginDate'] = pd.to_datetime(merged_data['BeginDate'], format='%Y-%m-%d', errors='coerce')

**Nota:** Aunque inicialmente parece que no hay valores ausentes en  merged_data, los valores "No" en la columna 'EndDate' no son reconocidos como ausentes(NaT) después de la conversión a tipo datetime, porque Python los trata como objetos. Por eso, es necesario reemplazar "No" manualmente por NaT y luego contar los valores ausentes. 

In [8]:
print("Valores ausentes en merged_data sin considerar 'No' en la columna 'EndDate'")
print(merged_data.isnull().sum())

Valores ausentes en merged_data sin considerar 'No' en la columna 'EndDate'
customerID          0
BeginDate           0
EndDate             0
Type                0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Gender              0
SeniorCitizen       0
Partner             0
Dependents          0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
MultipleLines       0
dtype: int64


In [9]:
# Reemplazar los valores ausentes(No) en la columna 'EndDate' por NaT(not a time).
merged_data['EndDate'] = merged_data['EndDate'].replace('No', pd.NaT)

# Cambiar tipo de datos en columna 'EndDate'(object) a datetime y con un formato que coincida con el dataset
merged_data['EndDate'] = pd.to_datetime(merged_data['EndDate'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

In [10]:
# Como puede observarse, la columna 'EndDate' es la única que tiene valores ausentes
print("Valores ausentes en merged_data después de cambiar 'No' por NaT en columna 'EndDate'")
print(merged_data.isnull().sum())

Valores ausentes en merged_data después de cambiar 'No' por NaT en columna 'EndDate'
customerID             0
BeginDate              0
EndDate             3249
Type                   0
PaperlessBilling       0
PaymentMethod          0
MonthlyCharges         0
TotalCharges           0
Gender                 0
SeniorCitizen          0
Partner                0
Dependents             0
InternetService        0
OnlineSecurity         0
OnlineBackup           0
DeviceProtection       0
TechSupport            0
StreamingTV            0
StreamingMovies        0
MultipleLines          0
dtype: int64


Inicialmente, se pensó en rellenar los valores nulos en la columna 'EndDate' con la fecha actual, pero no es una opción viable, ya que representaba la mayoría de los valores, lo que modificaría de manera sustancial el dataset y sesgaría los datos. 

In [11]:
# Crear una columna que almacene el estado original antes de la imputación
merged_data['OriginalChurn'] = merged_data['EndDate'].isna().astype(int)

In [12]:
# Agregar columna 'ContracDuration' que calcula la duración del contrato en días restando la diferencia entre las columnas 'BeginDate' y 'EndDate'
merged_data['ContractDuration'] = (merged_data['EndDate'] - merged_data['BeginDate']).dt.days

In [13]:
# Calcular la mediana de duración del contrato sin contar los valores ausentes (NaT)
median_duration = merged_data['ContractDuration'].dropna().median()
print(f"Duración mediana: {median_duration} días")

# Imputar valores ausentes en columna 'EndDate' sumando los valores datatime de 'BeginDate' + la mediana de la duración de días del contrato
merged_data['EndDate'] = merged_data['EndDate'].fillna(merged_data['BeginDate'] + timedelta(days=median_duration))

# Imputar valores ausentes en 'ContractDuration' con la mediana
merged_data['ContractDuration'] = merged_data['ContractDuration'].fillna(median_duration)

Duración mediana: 334.0 días


In [14]:
# Verificar que no hay valores ausentes dataset merged_data
print(merged_data.isna().sum())

customerID          0
BeginDate           0
EndDate             0
Type                0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Gender              0
SeniorCitizen       0
Partner             0
Dependents          0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
MultipleLines       0
OriginalChurn       0
ContractDuration    0
dtype: int64


In [15]:
# Imprimir 20 filas aleatorias después de aplicar los cambios
merged_data.sample(20)

,customerID,BeginDate,EndDate,Type,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Gender,SeniorCitizen,...,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,MultipleLines,OriginalChurn,ContractDuration
1015,2480-EJWYP,2015-04-01,2016-02-29,One year,Yes,Bank transfer (automatic),97.80,5458.8,Female,1,...,Fiber optic,No,Yes,No,No,Yes,Yes,Yes,1,334.0
4338,3736-BLEPA,2017-10-01,2018-08-31,One year,Yes,Bank transfer (automatic),91.25,2483.65,Male,0,...,Fiber optic,No,Yes,No,Yes,Yes,No,No,1,334.0
2654,4626-OZDTJ,2015-11-01,2016-09-30,One year,Yes,Credit card (automatic),78.65,3950.85,Female,0,...,DSL,No,Yes,No,Yes,Yes,Yes,Yes,1,334.0
1994,3617-XLSGQ,2014-08-01,2015-07-01,Two year,No,Bank transfer (automatic),109.25,7082.5,Female,0,...,Fiber optic,Yes,Yes,Yes,No,Yes,Yes,Yes,1,334.0
575,3196-NVXLZ,2018-01-01,2018-12-01,Month-to-month,Yes,Mailed check,84.60,2088.05,Female,0,...,Fiber optic,No,No,No,No,Yes,No,Yes,1,334.0
4722,3191-CSNMG,2019-11-01,2020-09-30,Month-to-month,Yes,Electronic check,80.70,239.45,Female,0,...,Fiber optic,No,No,No,No,Yes,No,No,1,334.0
321,0880-TKATG,2019-07-01,2019-11-01,Month-to-month,No,Electronic check,101.15,385.9,Male,0,...,Fiber optic,No,No,Yes,No,Yes,Yes,Yes,0,123.0
4730,1956-YIFGE,2018-04-01,2019-03-01,One year,Yes,Mailed check,100.05,2090.25,Male,0,...,Fiber optic,No,Yes,Yes,No,Yes,Yes,No,1,334.0
1970,2454-RPBRZ,2018-10-01,2019-10-01,Month-to-month,Yes,Electronic check,98.10,1060.2,Female,1,...,Fiber optic,No,Yes,Yes,No,Yes,No,Yes,0,365.0
1794,3227-WLKLI,2014-09-01,2015-08-01,Two year,No,Bank transfer (automatic),91.55,5963.95,Female,0,...,DSL,Yes,Yes,Yes,Yes,Yes,Yes,Yes,1,334.0


In [16]:
# Cambio de datos en columna TotalCharges(object) a tipo float
merged_data['TotalCharges'] = pd.to_numeric(merged_data['TotalCharges'], errors='coerce')

In [17]:
# Verificar cuántos valores ausentes hay en la columna TotalCharges
print("Valores ausentes en columna TotalCharges antes de eliminar:")
print(merged_data['TotalCharges'].isna().sum())

# Eliminar valores ausentes al no ser una proporción significativa de los datos
merged_data = merged_data.dropna(subset=['TotalCharges'])

# Verificar si quedan valores ausentes en TotalCharges
print("Valores ausentes en TotalCharges después de eliminar:")
print(merged_data['TotalCharges'].isna().sum())

# Revisar el nuevo tamaño del DataFrame
print("Número de filas después de eliminar valores ausentes:")
print(merged_data.shape[0])

# Restablecer el índice para que sea consecutivo
merged_data = merged_data.reset_index(drop=True)

Valores ausentes en columna TotalCharges antes de eliminar:
3
Valores ausentes en TotalCharges después de eliminar:
0
Número de filas después de eliminar valores ausentes:
4832


In [18]:
# Cambio de datos en columnas de tipo (object): Type, Gender, PaperlessBilling, PaymentMethod, Partner, Dependents, InternetService, OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV, StreamingMovies y MultipleLines a tipo categórico
categorical_cols = ['Type', 'Gender', 'PaperlessBilling', 'PaymentMethod', 'Partner', 
                    'Dependents', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 
                    'DeviceProtection', 'TechSupport', 'SeniorCitizen','StreamingTV', 'StreamingMovies', 
                    'MultipleLines']
merged_data[categorical_cols] = merged_data[categorical_cols].astype('category')

In [19]:
# Verificar tipo de datos en las columnas del dataset combinado después de la actualización de tipos de datos 
print(merged_data.dtypes)

customerID                  object
BeginDate           datetime64[ns]
EndDate             datetime64[ns]
Type                      category
PaperlessBilling          category
PaymentMethod             category
MonthlyCharges             float64
TotalCharges               float64
Gender                    category
SeniorCitizen             category
Partner                   category
Dependents                category
InternetService           category
OnlineSecurity            category
OnlineBackup              category
DeviceProtection          category
TechSupport               category
StreamingTV               category
StreamingMovies           category
MultipleLines             category
OriginalChurn                int32
ContractDuration           float64
dtype: object


In [20]:
# Verificar información general del dataset merged_data y contar valores ausentes
merged_data.info()
print()
print("Valores ausentes en merged_data:")
print(merged_data.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4832 entries, 0 to 4831
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   customerID        4832 non-null   object        
 1   BeginDate         4832 non-null   datetime64[ns]
 2   EndDate           4832 non-null   datetime64[ns]
 3   Type              4832 non-null   category      
 4   PaperlessBilling  4832 non-null   category      
 5   PaymentMethod     4832 non-null   category      
 6   MonthlyCharges    4832 non-null   float64       
 7   TotalCharges      4832 non-null   float64       
 8   Gender            4832 non-null   category      
 9   SeniorCitizen     4832 non-null   category      
 10  Partner           4832 non-null   category      
 11  Dependents        4832 non-null   category      
 12  InternetService   4832 non-null   category      
 13  OnlineSecurity    4832 non-null   category      
 14  OnlineBackup      4832 n

## Conclusiones de limpieza y preparación de datos

Se cargaron los datos localmente en una ruta igual al path de la plataforma ('/proyecto/datasets/final_provider/') asegurando accesibilidad para los revisores.  

Se combinaron los datasets contract, personal, phone e internet con el nombre merged_data, empleando la columna 'customerID' como clave primaria, e implementando el tipo de union inner join.    
Se formateó con CamelCase a las columnas para dar consistencia al nuevo dataset merged_data.  

Inicialmente, los valores nulos de 'EndDate' se imputaron con la fecha actual, pero esta práctica fue descartada para evitar sesgos significativos debido a la alta proporción de valores ausentes.
Las columnas 'EndDate' y 'BeginDate' fueron convertidas al formato datetime. Los valores nulos en 'BeginDate' se imputaron utilizando la mediana de 'ContractDuration' para evitar el impacto de outliers.

Se agregó la columna 'OriginalChurn' tomando la información de la columna 'EndDate' de merged_data. Explorando si tenía valores nulos y asignando True si el valor es ausente, posteriormente los valores True o False se convertían en enteros siendo True 1 y False 0 con astype(int)
  
Se cambió el tipo de datos en la columna 'TotalCharges' de object a float, y al solamente haber 3 valores ausentes se eliminaron al no tener una signifancia representativa en el análisis.  

Las columnas categóricas (Type, Gender, PaperlessBilling, etc.) fueron transformadas a tipo categórico y agrupadas en la variable categorical_cols.

Se verificaron los cambios y número de valores ausentes en todas las columnas con tipo de dato corregido.

In [22]:
# Análisis inicial de métricas estadísticas de las columnas numéricas del dataset
merged_data.describe()

,BeginDate,EndDate,MonthlyCharges,TotalCharges,OriginalChurn,ContractDuration
count,4832,4832,4832.000000,4832.000000,4832.000000,4832.000000
mean,2017-04-04 15:13:24.635761664,2018-05-22 21:24:44.105960448,81.761207,2902.470788,0.671772,413.257864
min,2013-10-01 00:00:00,2015-01-01 00:00:00,42.900000,42.900000,0.000000,30.000000
25%,2015-05-01 00:00:00,2016-06-30 00:00:00,69.787500,659.600000,0.000000,334.000000
50%,2017-07-01 00:00:00,2019-06-15 12:00:00,82.500000,2350.625000,1.000000,334.000000
75%,2019-03-01 00:00:00,2019-12-01 00:00:00,95.700000,4871.150000,1.000000,334.000000
max,2020-01-01 00:00:00,2020-11-30 00:00:00,118.750000,8684.800000,1.000000,2191.000000
std,NaN,NaN,18.306134,2415.564128,0.469617,362.622174


Puede observarse que el conjunto de datos consta de 4832 filas.  

Las medias en las columnas 'BeginDate' y 'EndDate' son abril de 2017 y mayo de 2018 respectivamente, mientras que ambas presentan una variación estándar con NaN por ser datos tipo datetime.  Lo que sugiere que la fecha de inicio de contrato es más homogénea que la fecha de término, mosntrando una tendencia hacia contratos más cortos.

En la columna 'MonthlyCharges' se presenta una inclinación hacia la derecha, es decir con valores ligeramente más cercanos al máximo. También tiene una variación estándar considerable de 18.30.  Lo que podría indicar que las tarifas mensuales son ligeramente más elevadas que el gasto promedio.  

Respecto a 'TotalCharges' hay una marcada inclinación hacia la izquierda, más cercana a los valores mínimos y es la característica con mayor varición estándar con 2415.56. Lo que sugiere que los clientes pagan cantidades más cercanas a los 42 dólares (mínimo).  

En cuanto a 'ContractDuration' resulta evidente una inclinación hacia la izquierda, más cercanas a los valores mínimos, Observando que la duración de los contratos tienen tendencia a ser más cortos.